# Time travel
Using the rank cluster, we can run our collection of real search terms against the new mapping with the new query structure. We can then analyse the results according to the same set of metrics as we used for the data which was collected in real time. In other words, we can look at how search _would have_ performed if we had made these changes earlier. It's a time-travelling A/B test.

In [ ]:
import os
import json
from elasticsearch import Elasticsearch

## Getting queries
Because the queries are written and tested in typescript, we need to export a json version of them before they can be used in these python notebooks. Running `yarn getQueryJSON <query_name>` will generate a `.json` version of the query alongside the `.ts` original.

We can then import the query as follows:

In [ ]:
query_name = "works-with-search-fields"

In [ ]:
with open(f"data/queries/{query_name}.json", "r") as f:
    query = json.load(f)

query

we can now open a connection to our rank cluster and run our query against it

In [ ]:
secret = lambda name: os.environ[name][1:-1]

es = Elasticsearch(
    cloud_id=secret("ES_RANK_CLOUD_ID"),
    http_auth=(
        secret("ES_RANK_USER"), 
        secret("ES_RANK_PASSWORD")
    ),
)

es.indices.exists(index=query_name)

In [ ]:
def format_query(search_term): 
    return {"query": json.loads(json.dumps(query).replace("{{query}}", search_term))}

In [ ]:
es.search(
    index=query_name,
    body=format_query("skeleton")
)